In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt

# -------------------------------
# Objective functions
# -------------------------------
def test_problem1(x):
    # Accepts x as a list or tuple [x1, x2]
    x1, x2 = x
    return 100 * (x1**2 - x2)**2 + (1 - x1) ** 2

def rastrigin(x):
    A = 10
    n = len(x)
    return A * n + sum([(xi**2 - A * np.cos(2 * np.pi * xi)) for xi in x])
# -------------------------------
# Encoding helpers
# -------------------------------
def decode_binary(bits, bounds, precision=4):
    # Decode bitstring to float with given bounds
    l, u = bounds
    intval = int("".join(str(b) for b in bits), 2)
    max_int = 2**len(bits) - 1
    return l + (u - l) * intval / max_int

def encode_binary(x, bounds, n_bits):
    l, u = bounds
    intval = int(round((x - l) / (u - l) * (2**n_bits - 1)))
    return list(bin(intval)[2:].zfill(n_bits))

# -------------------------------
# Selection
# -------------------------------
def roulette_wheel_selection(fitness):
    max_val = sum(fitness)
    pick = random.uniform(0, max_val)
    current = 0
    for i, f in enumerate(fitness):
        current += f
        if current > pick:
            return i

def tournament_selection(pop, fitness):
    i, j = random.sample(range(len(pop)), 2)
    return i if fitness[i] < fitness[j] else j

# -------------------------------
# Crossover
# -------------------------------
def single_point_crossover(p1, p2):
    if len(p1) <= 1:
        return p1[:], p2[:]
    point = random.randint(1, len(p1)-1)
    return p1[:point] + p2[point:], p2[:point] + p1[point:]

def simulated_binary_crossover(x1, x2, eta=20):
    if random.random() > 0.9:
        return x1, x2
    c1, c2 = [], []
    for i in range(len(x1)):
        u = random.random()
        if u <= 0.5:
            beta = (2 * u) ** (1.0 / (eta + 1))
        else:
            beta = (1.0 / (2.0 * (1.0 - u))) ** (1.0 / (eta + 1))
        c1.append(0.5 * ((1 + beta) * x1[i] + (1 - beta) * x2[i]))
        c2.append(0.5 * ((1 - beta) * x1[i] + (1 + beta) * x2[i]))
    return c1, c2

# -------------------------------
# Mutation
# -------------------------------
def binary_mutation(bitstring, pm):
    return [str(1-int(b)) if random.random() < pm else b for b in bitstring]

def polynomial_mutation(x, bounds, eta=20, pm=None):
    y = []
    n = len(x)
    if pm is None:
        pm = 1.0 / n
    for i in range(n):
        if random.random() < pm:
            l, u = bounds[i]
            delta1 = (x[i] - l) / (u - l)
            delta2 = (u - x[i]) / (u - l)
            rand = random.random()
            mut_pow = 1.0 / (eta + 1.)
            if rand < 0.5:
                xy = 1.0 - delta1
                val = 2.0 * rand + (1.0 - 2.0 * rand) * (xy ** (eta + 1))
                deltaq = val**mut_pow - 1.0
            else:
                xy = 1.0 - delta2
                val = 2.0 * (1.0 - rand) + 2.0 * (rand - 0.5) * (xy ** (eta + 1))
                deltaq = 1.0 - val**mut_pow
            xi = x[i] + deltaq * (u - l)
            xi = min(max(xi, l), u)
            y.append(xi)
        else:
            y.append(x[i])
    return y

# -------------------------------
# Genetic Algorithm
# -------------------------------
def genetic_algorithm(func, bounds, n_var, encoding="real", n_bits=16, pop_size=50, generations=100):
    # Initialize population
    if encoding == "binary":
        chrom_length = n_bits * n_var
        population = [[str(random.randint(0,1)) for _ in range(chrom_length)] for _ in range(pop_size)]
    else:
        population = [np.random.uniform([b[0] for b in bounds], [b[1] for b in bounds]).tolist() for _ in range(pop_size)]

    best_per_gen = []

    for gen in range(generations):
        # Decode & fitness
        if encoding == "binary":
            decoded = []
            for chrom in population:
                vars_ = []
                for i in range(n_var):
                    bits = chrom[i*n_bits:(i+1)*n_bits]
                    vars_.append(decode_binary(bits, bounds[i]))
                decoded.append(vars_)
        else:
            decoded = population

        fitness = [func(ind) for ind in decoded]
        best_idx = np.argmin(fitness)
        best_per_gen.append(fitness[best_idx])

        # New population
        new_pop = []
        while len(new_pop) < pop_size:
            if encoding == "binary":
                p1 = population[roulette_wheel_selection([1/(f+1e-6) for f in fitness])]
                p2 = population[roulette_wheel_selection([1/(f+1e-6) for f in fitness])]
                if random.random() < 0.9:
                    c1, c2 = single_point_crossover(p1, p2)
                else:
                    c1, c2 = p1[:], p2[:]
                c1 = binary_mutation(c1, 1.0/(n_bits*n_var))
                c2 = binary_mutation(c2, 1.0/(n_bits*n_var)) 
                new_pop.extend([c1,c2])
            else:
                p1 = population[tournament_selection(population, fitness)]
                p2 = population[tournament_selection(population, fitness)]
                c1, c2 = simulated_binary_crossover(p1, p2)
                c1 = polynomial_mutation(c1, bounds)
                c2 = polynomial_mutation(c2, bounds)
                new_pop.extend([c1,c2])
        population = new_pop[:pop_size]
    return best_per_gen




In [11]:
# Example for Test Problem 1, binary encoding
results_bin = []
results_real = []
generations = 50  # Set based on convergence plot

for _ in range(20):
    res_bin = genetic_algorithm(test_problem1, [(-2.048, 2.048), (-2.048, 2.048)], n_var=2, encoding="binary", generations=generations)
    results_bin.append(res_bin[-1])  # Best fitness at last generation
    res_real = genetic_algorithm(test_problem1, [(-2.048, 2.048), (-2.048, 2.048)], n_var=2, encoding="real", generations=generations)
    results_real.append(res_real[-1])

print("Binary encoding: mean =", np.mean(results_bin), "std =", np.std(results_bin), "min =", np.min(results_bin), "max =", np.max(results_bin))
print("Real encoding: mean =", np.mean(results_real), "std =", np.std(results_real), "min =", np.min(results_real), "max =", np.max(results_real))
# ...repeat for rastrigin n=2 and n=5...

Binary encoding: mean = 0.07167321196122756 std = 0.07022130371600514 min = 0.004258740525158702 max = 0.2731573765598866
Real encoding: mean = 0.11195542117918009 std = 0.13072129449677283 min = 0.000546303411088541 max = 0.5027282129642552


In [13]:
results_real

[0.00540236624557322,
 0.0083152441673386,
 0.014834409728172779,
 0.37519015065294054,
 0.012753084924618182,
 0.22364375296464717,
 0.5027282129642552,
 0.17053652066993827,
 0.0021540093843978995,
 0.07499099197443448,
 0.000546303411088541,
 0.06926266807583098,
 0.09905190439080636,
 0.24020517163477567,
 0.1006119996827046,
 0.056974234132951156,
 0.05382014229952631,
 0.0904503953445682,
 0.13007001977404797,
 0.007566841160985629]

In [4]:
# -------------------------------
# Example run
# -------------------------------
if __name__ == "__main__":
    bounds_test1 = [(-2.048, 2.048), (-2.048, 2.048)]
    res_bin = genetic_algorithm(test_problem1, bounds_test1, n_var=2, encoding="binary", generations=50)
    res_real = genetic_algorithm(test_problem1, bounds_test1, n_var=2, encoding="real", generations=50)
    plt.plot(res_bin, label="Binary")
    plt.plot(res_real, label="Real")
    plt.xlabel("Generations")
    plt.ylabel("Best fitness")
    plt.legend()
    plt.show()

TypeError: test_problem1() missing 1 required positional argument: 'x2'